<a href="https://colab.research.google.com/github/xinyanz-erin/Applied-Finance-Project/blob/Pui/European_Call_jax.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [36]:
import jax.numpy as jnp
from jax import random
from jax import jit
from jax import device_put

def test():
  paths=100000
  steps=252
  stocks=3
  key = random.PRNGKey(10)
  x = random.normal(key, (paths,steps,stocks))
  s0=random.randint(key,(stocks,1),50,100)*1.
  s=s0
  s = device_put(s)
  for i in range(steps):
    for j in range(stocks):
      s+=x[:,i,j]
  return(s0,s)

a,b=test()

print(a,np.mean(b,axis=1))

jtest=jit(test)

c,d=jtest()

print(c,np.mean(d,axis=1))

[[86.]
 [56.]
 [71.]] [85.99678  55.996777 70.99677 ]
[[86.]
 [56.]
 [71.]] [85.99678  55.996777 70.99677 ]


In [37]:
paths=10
steps=5
stocks=3
key = random.PRNGKey(10)
x = random.normal(key, (paths,steps,stocks))
s0=random.randint(key,(stocks,1),50,100)*1.
s=s0
s = device_put(s)
print(s)
print(x[:,1,1])
print(s+x[:,1,1])

[[86.]
 [56.]
 [71.]]
[-0.32907748 -0.3529078  -0.5357338  -2.402825   -1.3187171  -0.4300647
 -0.42437345  0.22680947  0.3185347  -0.9541187 ]
[[85.67092  85.647095 85.464264 83.597176 84.68128  85.56994  85.57563
  86.22681  86.318535 85.04588 ]
 [55.67092  55.64709  55.464268 53.597176 54.681282 55.569935 55.575626
  56.22681  56.318535 55.045883]
 [70.67092  70.647095 70.464264 68.597176 69.68128  70.56994  70.57563
  71.22681  71.318535 70.04588 ]]


In [65]:
import jax.numpy as jnp
def simple_process(key, initial_values, numsteps, drift, cov):
    stocks_init = jnp.zeros((numsteps + 1,initial_values.shape[0]))
    stocks_init=jax.ops.index_update(stocks_init,  # jax.ops.index_update(x, idx, y) <-> Pure equivalent of x[idx] = y
                            jax.ops.index[0],
                            initial_values)
    noise = jax.random.multivariate_normal(key, drift, cov, (numsteps,))
    #return(noise)
    def time_step(t, val):
        dx =  drift+noise[t,:]
        val = jax.ops.index_update(val,
                            jax.ops.index[t],
                            val[t-1] + dx)
        return val
    return jax.lax.fori_loop(1, numsteps+1, time_step, stocks_init)[1:] # jax.lax.fori_loop(lower, upper, body_fun, init_val)

# def fori_loop(lower, upper, body_fun, init_val): # upper is exclusive
#   val = init_val
#   for i in range(lower, upper):
#     val = body_fun(i, val)
#   return val

numsteps=5
key = random.PRNGKey(10)
drift=jnp.array([0.0]*3)
cov=np.random.random((3,3))
cov=np.matmul(cov,cov.T)
initial_values=jnp.array([100.]*3)
fast_simple = jax.jit(simple_process, static_argnums=2)
init_stocks=jnp.array([100.]*3)
fast_simple(key,init_stocks,numsteps,drift,cov)
# Batch OU sample via vmap
batch_simple = jax.vmap(fast_simple, in_axes=(0, None, None, None, None)) 
# An integer or None indicates which array axis to map over for all arguments (with None indicating not to map any axis)

%timeit fast_simple(key, init_stocks, 12, drift, cov)

numsamples=100000 # num of paths
keys = jax.random.split(key, numsamples)
batch_simple = jax.vmap(fast_simple, in_axes=(0, None, None, None, None))
%timeit batch_simple(keys, init_stocks, 12, drift, cov)

batch_simple(keys, init_stocks, numsteps, drift, cov).shape

The slowest run took 45672.07 times longer than the fastest. This could mean that an intermediate result is being cached.
1 loop, best of 5: 10.2 µs per loop
1 loop, best of 5: 184 ms per loop


(100000, 5, 3)

In [52]:
key = random.PRNGKey(10)
initial_values=jnp.array([100.]*3)
numsteps=5
drift=jnp.array([0.0]*3)
cov=np.random.random((3,3))
cov=np.matmul(cov, cov.T)

print(key)
print(initial_values)
print(numsteps)
print(drift)
print(cov)

[ 0 10]
[100. 100. 100.]
5
[0. 0. 0.]
[[1.54747825 0.99664178 0.78596654]
 [0.99664178 0.71180915 0.49595259]
 [0.78596654 0.49595259 0.81469483]]


In [49]:
stocks_init = jnp.zeros((numsteps + 1, initial_values.shape[0]))
print(stocks_init)
stocks_init=jax.ops.index_update(stocks_init,
                        jax.ops.index[0],
                        initial_values)
print(stocks_init)
noise = jax.random.multivariate_normal(key, drift, cov, (numsteps,))
print(noise)

[[0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]]
[[100. 100. 100.]
 [  0.   0.   0.]
 [  0.   0.   0.]
 [  0.   0.   0.]
 [  0.   0.   0.]
 [  0.   0.   0.]]
[[-0.04484753 -0.9199365   1.5313144 ]
 [ 0.5629996  -0.19369014 -0.08328605]
 [ 0.09276688 -0.04558031 -0.5952471 ]
 [ 0.3374979  -0.36135596 -1.0123413 ]
 [-0.19334996 -0.20442696 -0.20229332]]
